In [ ]:
import autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

## Configs

In [ ]:
BATCH_SIZE = 32

## Preprocessing
Negative Sampling the dataset

In [ ]:
from src.Preprocess.Preprocessor import Preprocessor
Preprocessor.create_negative_sampled_dataset(input_address="../data/SA_history_clicks_of_98.csv", 
                                               output_address="../data/data1.pkl", 
                                               neg_to_pos_ratio=1)

## DatasetHandler
Preparing dataset for training

In [1]:
from src.DatasetHandler import DatasetHandler
# dataset_handler = DatasetHandler("data/SA_history_clicks_of_98.csv")
dataset_handler = DatasetHandler("../data/small1.pkl")

Loading tokenizer...
Reading input file...



Creating dataset...



Done


In [2]:
import pandas as pd
# dataset_handler.df.loc[dataset_handler.df["similar"] == 0]
print(dataset_handler.get_dataset())
dataset_handler.df
pd.DataFrame(dataset_handler.get_dataset()["train"][:1000])

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'attention_mask', 'input_ids', 'packageName', 'package_ids', 'queryText', 'similar', 'token_type_ids'],
        num_rows: 17516
    })
    test: Dataset({
        features: ['__index_level_0__', 'attention_mask', 'input_ids', 'packageName', 'package_ids', 'queryText', 'similar', 'token_type_ids'],
        num_rows: 4380
    })
})


,__index_level_0__,attention_mask,input_ids,packageName,package_ids,queryText,similar,token_type_ids
0,11574,"[1, 1, 1, 0, 0, 0, 0, 0]","[2, 6483, 4, 0, 0, 0, 0, 0]",app.womostore.ir,162667,کیش,-1,"[0, 0, 0, 0, 0, 0, 0, 0]"
1,19630,"[1, 1, 1, 1, 1, 1, 0, 0]","[2, 3879, 5054, 23951, 6449, 4, 0, 0]",com.dibmehr.uninstallapp,141673,نرم افزار نرمش بانوان,-1,"[0, 0, 0, 0, 0, 0, 0, 0]"
2,7305,"[1, 1, 1, 0, 0, 0, 0, 0]","[2, 45726, 4, 0, 0, 0, 0, 0]",com.ovilex.extremetruckssimulator,144576,کمپرسی,1,"[0, 0, 0, 0, 0, 0, 0, 0]"
3,19584,"[1, 1, 1, 1, 0, 0, 0, 0]","[2, 18882, 9027, 4, 0, 0, 0, 0]",iran.shantya.texttofilm,124450,واتر مارک,1,"[0, 0, 0, 0, 0, 0, 0, 0]"
4,7924,"[1, 1, 1, 1, 1, 1, 1, 1]","[2, 14790, 3922, 7749, 4175, 2754, 14236, 4]",com.shake_se.live_wallpaper.fateme_livewallpaper,151570,ولادت حضرت فاطمه تصویر زمینه متحرک,1,"[0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...
995,8057,"[1, 1, 1, 0, 0, 0, 0, 0]","[2, 4211, 4, 0, 0, 0, 0, 0]",ir.raikaapp.msa,146653,صدا,-1,"[0, 0, 0, 0, 0, 0, 0, 0]"
996,4805,"[1, 1, 1, 1, 1, 0, 0, 0]","[2, 6316, 9333, 3190, 4, 0, 0, 0]",com.rozblack.telegramexplorer,110609,رها منسجر,1,"[0, 0, 0, 0, 0, 0, 0, 0]"
997,11954,"[1, 1, 1, 1, 0, 0, 0, 0]","[2, 15089, 17201, 4, 0, 0, 0, 0]",com.jb.gokeyboard.theme.tmekeyboardthemeforand...,144740,کیبوردرنگی,1,"[0, 0, 0, 0, 0, 0, 0, 0]"
998,12831,"[1, 1, 1, 1, 1, 0, 0, 0]","[2, 4679, 1154, 8642, 4, 0, 0, 0]",com.mobopic.android,100189,موبو پیک,1,"[0, 0, 0, 0, 0, 0, 0, 0]"


## Training

In [ ]:
import torch
from src.SpanPooling import *

class RepresentationBuilder(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim=300):
        super(RepresentationBuilder, self).__init__()
        self.embedding = torch.nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.pooling_module = get_pooling_module("avg")
        
    def forward(self, input_ids, attention_mask):
        out = self.embedding(input_ids)
        print(out)
        out = self.pooling_module(out, attention_mask)
        return out


In [ ]:
import torch

class Siamese(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Siamese, self).__init__()
        self.representation_builder = torch.nn.Sequential(
            RepresentationBuilder(vocab_size, embedding_dim),
            torch.nn.ReLU()
        )
        self.loss = torch.nn.CosineEmbeddingLoss(margin=0)
        
    def forward(self, x1, x2):
        x1_repr = self.representation_builder(x1)
        x2_repr = self.representation_builder(x2)
        print(x1_repr)
        print(x2_repr)
        print(self.loss(x1_repr, x2_repr, target=torch.tensor([-1, 1])))
        

In [ ]:
import torch
from src.DatasetHandler import DatasetHandler

class Trainer:
    def __init__(self, vocab_size, embedding_dim, dataset_handler: DatasetHandler):
        self.siamese = Siamese(vocab_size, embedding_dim)
    
    def train(self):
        self.siamese(torch.tensor([[1], [0, 0, 0, 0]]), torch.tensor([[1, 2, 3, 3], [9, 9, 9, 9]]))
        
    def predict_from_query(self, queryText):
        

In [ ]:
# dataset_handler.df[:100]
# dataset_handler.dataset
# d = dataset_handler.df.to_dict()

In [ ]:
dd = dataset_handler.df.sample(frac=0.01)
dd.to_pickle("small.pkl")

In [ ]:
trainer = Trainer(vocab_size=10, embedding_dim=5, dataset_handler=dataset_handler)
trainer.train()